# 기본설정

In [1]:
# Google drive mout
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import math
import requests
import datetime, pytz
from bs4 import BeautifulSoup

In [3]:
mytz = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(mytz)

# print("현재 : ", now)
# print("현재 날짜 : ", now.date())
# print("현재 시간 : ", now.time())
# print("timestamp : ", now.timestamp())
# print("년 : ", now.year)
# print("월 : ", now.month)
# print("일 : ", now.day)
# print("시 : ", now.hour)
# print("분 : ", now.minute)
# print("초 : ", now.second)
# print("마이크로초 : ", now.microsecond)
# print("요일 : ", now.weekday())
# print("문자열 변환 : ", now.strftime('%Y-%m-%d %H:%M:%S'))

In [4]:
print("현재 날짜 : ", now.date())
print("현재 시간 : ", now.time())
print("현재 날짜, 시간 : ", now.strftime('%Y-%m-%d %H:%M:%S'))

현재 날짜 :  2023-01-04
현재 시간 :  20:48:12.852659
현재 날짜, 시간 :  2023-01-04 20:48:12


# 콘솔 리스트 만들기

In [5]:
# console list

url = "https://www.vgchartz.com/games/games.php?page=1&order=Sales&ownership=Both&direction=DESC&showtotalsales=0&shownasales=0&showpalsales=0&showjapansales=0&showothersales=0&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=0&showcriticscore=0&showuserscore=0&showshipped=0"
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

console = soup.find('select', attrs={"name":"console"})

console_list = []
for i in console.find_all('option')[1:]:
  console_list.append(i.string)

In [6]:
console_list = pd.DataFrame(console_list) # 데이터프레임 저장
console_list_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/console_list.csv'
console_list.to_csv(console_list_path, sep=',', encoding = 'utf-8')
print("Save Date / Time : ", now.strftime('%Y-%m-%d %H:%M:%S'))

Save Date / Time :  2023-01-04 20:48:12


In [7]:
console_list

0
0   3DO Interactive Multiplayer
1                Acorn Electron
2                           All
3                         Amiga
4                    Amiga CD32
..                          ...
85                     Xbox 360
86                    XBox Live
87                     Xbox One
88                  Xbox Series
89                  ZX Spectrum

[90 rows x 1 columns]

In [8]:
# console list2

url = "https://www.vgchartz.com/charts/platform_totals/Hardware.php/"
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

console_table = soup.find('table', attrs={"id":"myTable"})
console_table_tbody = console_table.tbody

console_list2 = []

for row in console_table_tbody.find_all('tr'):
  col = row.select('td')
  console_list2.append(col[1].string)


In [9]:
console_list2 = pd.DataFrame(console_list2) # 데이터프레임 저장
console_list2_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/console_list2.csv'
console_list2.to_csv(console_list2_path, sep=',', encoding = 'utf-8')

# 장르 리스트 만들기

In [10]:
# 장르 리스트 만들기
for page in range(1,2):
  url = "https://www.vgchartz.com/games/games.php?page={}&results=200&genre=Action&order=Sales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=1".format(page)
  res = requests.get(url)
  res.raise_for_status()
  soup = BeautifulSoup(res.text, "lxml")

  genre = soup.find('select', attrs={"name":"genre"})

genre_list = []
for i in genre.find_all('option')[1:]:
  genre_list.append(i.string)

In [11]:
print("Scrap Date / Time : ", now.strftime('%Y-%m-%d %H:%M:%S'))
genre_list

Scrap Date / Time :  2023-01-04 20:48:12


['Action',
 'Action-Adventure',
 'Adventure',
 'Board Game',
 'Education',
 'Fighting',
 'Misc',
 'MMO',
 'Music',
 'Party',
 'Platform',
 'Puzzle',
 'Racing',
 'Role-Playing',
 'Sandbox',
 'Shooter',
 'Simulation',
 'Sports',
 'Strategy',
 'Visual Novel']

In [12]:
# ['Action', 'Action-Adventure', 'Adventure', 'Board Game', 'Education', 'Fighting', 'Misc', 'MMO', 'Music', 'Party', 'Platform']
# ['Puzzle', 'Racing', 'Role-Playing', 'Sandbox', 'Shooter', 'Simulation', 'Sports', 'Strategy', 'Visual Novel']
# genre_list = ['Puzzle', 'Racing', 'Role-Playing', 'Sandbox', 'Shooter', 'Simulation', 'Sports', 'Strategy', 'Visual Novel']

In [13]:
# 빈컬럼 만들기
Genre = []
Name = []
g_url = []
Platform = []
Publisher = []
VGC_Score = []
Critic_Score = []
User_Score = []
Total_Sales = []
NA_Sales = []
PAL_Sales = []
JP_Sales = []
Other_Sales = []
Year = []

# 장르별 게임데이터 만들기

In [14]:
# 장르별 페이지 수 계산
for gen in genre_list:
  url = "https://www.vgchartz.com/games/games.php?page=1&results=200&genre={}&order=Sales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=1".format(gen)
  res = requests.get(url)
  res.raise_for_status()
  soup = BeautifulSoup(res.text, "lxml")

  chart = soup.select_one('div#generalBody > table')
  result_str = chart.find_all('tr')[0].th.string
  pages = math.floor(int(re.sub(r'[^0-9]', '', result_str[result_str.find('(')+1 : result_str.find(')')])) / 200) + 2
  print(gen, ' 📖 total pages : ', pages - 1)

# 스크래핑
  for page in range(1, pages):
    url = "https://www.vgchartz.com/games/games.php?page={}&results=200&genre={}&order=Sales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=0".format(page, gen)
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    chart = soup.select_one('div#generalBody > table')

    for row in chart.find_all('tr')[3:]:
      col = row.select('td')
      
      Genre.append(gen)
      Name.append(col[2].a.string)
      g_url.append(col[2].a['href'][:-11])
      Platform.append(col[3].find("img")['alt'])
      Publisher.append(col[4].string)

      VGC_Score.append(col[5].string)
      Critic_Score.append(col[6].string)
      User_Score.append(col[7].string)

      Total_Sales.append(col[8].string)
      NA_Sales.append(col[9].string)
      PAL_Sales.append(col[10].string)
      JP_Sales.append(col[11].string)
      Other_Sales.append(col[12].string)
      Year.append(col[13].string)

    print(type(chart), gen, '🚀 page number : ', page)

print("Scrap Date / Time : ", now.strftime('%Y-%m-%d %H:%M:%S'))

Action  📖 total pages :  42
<class 'bs4.element.Tag'> Action 🚀 page number :  1
<class 'bs4.element.Tag'> Action 🚀 page number :  2
<class 'bs4.element.Tag'> Action 🚀 page number :  3
<class 'bs4.element.Tag'> Action 🚀 page number :  4
<class 'bs4.element.Tag'> Action 🚀 page number :  5
<class 'bs4.element.Tag'> Action 🚀 page number :  6
<class 'bs4.element.Tag'> Action 🚀 page number :  7
<class 'bs4.element.Tag'> Action 🚀 page number :  8
<class 'bs4.element.Tag'> Action 🚀 page number :  9
<class 'bs4.element.Tag'> Action 🚀 page number :  10
<class 'bs4.element.Tag'> Action 🚀 page number :  11
<class 'bs4.element.Tag'> Action 🚀 page number :  12
<class 'bs4.element.Tag'> Action 🚀 page number :  13
<class 'bs4.element.Tag'> Action 🚀 page number :  14
<class 'bs4.element.Tag'> Action 🚀 page number :  15
<class 'bs4.element.Tag'> Action 🚀 page number :  16
<class 'bs4.element.Tag'> Action 🚀 page number :  17
<class 'bs4.element.Tag'> Action 🚀 page number :  18
<class 'bs4.element.Tag'> A

In [15]:
columns = {
  'Genre' : Genre,
  'Name' : Name,
  'url' : g_url,
  'Platform' : Platform,
  'Publisher' : Publisher,
  'VGC_Score' : VGC_Score,
  'Critic_Score' : Critic_Score,
  'User_Score' : User_Score,
  'Total_Sales' : Total_Sales,
  'NA_Sales' : NA_Sales,
  'PAL_Sales' : PAL_Sales,
  'JP_Sales' : JP_Sales,
  'Other_Sales' : Other_Sales,
  'Year' : Year
  }

In [16]:
VGC_df = pd.DataFrame(columns) # 데이터프레임 저장
VGC_df_path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/vgchartz.csv'
VGC_df.to_csv(VGC_df_path, sep=',', encoding = 'utf-8')
print("Save Date / Time : ", now.strftime('%Y-%m-%d %H:%M:%S'))

Save Date / Time :  2023-01-04 20:48:12
